# Predicting Song Listens Using Apache Spark

One of the most common uses of big data is to predict what users want.  This allows Google to show you relevant ads, Amazon to recommend relevant products, and Netflix to recommend movies that you might like. We will start with some basic techniques, and then use the [Spark ML][sparkml] library's Alternating Least Squares method to make more sophisticated predictions.

We use triplets from 1 million users from the [million song dataset](http://labrosa.ee.columbia.edu/millionsong/pages/getting-dataset), along with metadata for the songs that includes information such as the song title, artist name and tempo. The triplets com in the form (User ID, Song ID, Number of Plays). 
[sparkml]: https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html

### Load and Cache

The Databricks File System (DBFS) sits on top of S3.  We use both the song plays csv and the song metadata in memory.

In [3]:
# import libraries
import os
from pyspark.sql.types import *
from pyspark.sql import functions as F

# list directories
base_dir = '/FileStore/'
triplets_filename = base_dir + 'train_triplets.txt'

#base_dir2 = '/FileStore/tables/qgz865tr1472080731816/'
songs2tracks_filename = base_dir + 'taste_profile_song_to_tracks.txt'

#base_dir3 = '/FileStore/tables/9204dwdp1472230135186/'
metadata_filename = base_dir + 'track_metadata.csv'

#lyrics_file = base_dr + 'lyrics.txt'

if os.path.sep != '/':
  # Handle Windows.
  triplets_filename = triplets_filename.replace('/', os.path.sep)
  songs2tracks_filename = songs2tracks_filename.replace('/', os.path.sep)
  metadata_filename = metadata_filename.replace('/', os.path.sep)
  #lyrics_file = lyrics_file.replace('/', os.path.sep)
  

In [4]:
# Create schema so the cluster only runs through the data once
plays_df_schema = StructType(
  [StructField('userId', StringType()),
   StructField('songId', StringType()),
   StructField('Plays', IntegerType())]
)

songs2tracks_df_schema = StructType(
  [StructField('songId', StringType()),
   StructField('trackId', StringType())]
)

metadata_df_schema = StructType(
  [StructField('', IntegerType()),
   StructField('analyzer_version', IntegerType()),
   StructField('artist_7digitalid', IntegerType()),
   StructField('artist_familiarity', DoubleType()),
   StructField('artist_hotttness', DoubleType()),
   StructField('artist_id', StringType()),
   StructField('artist_latitude', DoubleType()),
   StructField('artist_location', StringType()),
   StructField('artist_longitude', DoubleType()),
   StructField('artist_mbid', StringType()),
   StructField('artist_name', StringType()),
   StructField('artist_playmeid', IntegerType()),
   StructField('genre', StringType()),
   StructField('idx_artist_terms', IntegerType()),
   StructField('idx_similar_artists', IntegerType()),
   StructField('release', StringType()),
   StructField('release_7digitalid', IntegerType()),
   StructField('song_hotttnesss', DoubleType()),
   StructField('songId', StringType()),
   StructField('title', StringType()),
   StructField('track_7digitalid', IntegerType())
   #StructField('index', IntegerType()),
   #StructField('trackId', StringType()),
   #StructField('duration', DoubleType()),
   #StructField('year', IntegerType()),
   #StructField('shs_perf', DoubleType()),
   #StructField('shs_work', DoubleType())
  ]
)

In [5]:
# load in data
raw_plays_df = sqlContext.read.format('com.databricks.spark.csv') \
                              .options(delimiter = '\t', header=True,inferSchema=False) \
                              .schema(plays_df_schema) \
                              .load(triplets_filename)

songs2tracks_df = sqlContext.read.format('com.databricks.spark.csv') \
                                 .options(delimiter = '\t', header=True,inferSchema=False) \
                                 .schema(songs2tracks_df_schema) \
                                 .load(songs2tracks_filename)

metadata_df = sqlContext.read.format('com.databricks.spark.csv') \
                             .options(delimiter = ',', header=True,inferSchema=False) \
                             .schema(metadata_df_schema) \
                             .load(metadata_filename)

#lyrics_df = sqlContext.read.format('com.databricks.spark.csv') \
#                             .options(delimiter = ',', header=True,inferSchema=True) 
#                            

In [6]:
from pyspark.sql.functions import avg
display(metadata_df.select("artist_familiarity","artist_hotttness","artist_name").groupBy("artist_name").agg(avg("artist_familiarity"),avg("artist_hotttness")))

artist_name,avg(artist_familiarity),avg(artist_hotttness)
Eurythmics,0.7510076200369779,0.5236990777653908
Pery Ribeiro,0.38541744679271267,0.31213122149558203
Silverstein,0.7975761098264473,0.6055071356905753
Michelle & Vickie,0.36840432673225254,0.30379069801488423
Marga Gomez,0.3107765232758942,0.22842866681038246
DJ Taz,0.4099894915499974,0.240551990468017
Rhian Sheehan,0.5580077519194866,0.41478065907349704
Brian Tyler,0.6752954694563452,0.4426967195814664
Faith Yang,0.37402463303512845,0.18721329024920533
Generation X,0.6111809474469422,0.39903376959197284


In [7]:
# cache the dataframes
raw_plays_df.cache()
raw_plays_df.show(5)

songs2tracks_df.cache()
songs2tracks_df.show(5)

metadata_df.cache()
metadata_df.show(5)

+--------------------+------------------+-----+
 userId| songId|Plays|
+--------------------+------------------+-----+
b80344d063b5ccb32...|SOAPDEY12A81C210A9| 1|
b80344d063b5ccb32...|SOBBMDR12A8C13253B| 2|
b80344d063b5ccb32...|SOBFNSP12AF72A0E22| 1|
b80344d063b5ccb32...|SOBFOVM12A58A7D494| 1|
b80344d063b5ccb32...|SOBNZDC12A6D4FC103| 1|
+--------------------+------------------+-----+
only showing top 5 rows

+------------------+------------------+
 songId| trackId|
+------------------+------------------+
SOAAADE12A6D4F80CC|TRSKKFK128F148B615|
SOAAADF12A8C13DF62|TRCQMSP128F428A6F7|
SOAAADZ12A8C1334FB|TRMDNZY128F425A532|
SOAAAFI12A6D4F9C66|TRZEXLQ128F1491D17|
SOAAAGK12AB0189572|TRDZUFJ12903CE29FC|
+------------------+------------------+
only showing top 5 rows

+---+----------------+-----------------+------------------+-------------------+------------------+---------------+---------------+----------------+--------------------+----------------+---------------+-----+----------------+-------------------+--------------------+------------------+------------------+------------------+----------------+----------------+
 |analyzer_version|artist_7digitalid|artist_familiarity| artist_hotttness| artist_id|artist_latitude|artist_location|artist_longitude| artist_mbid| artist_name|artist_playmeid|genre|idx_artist_terms|idx_similar_artists| release|release_7digitalid| song_hotttnesss| songId| title|track_7digitalid|
+---+----------------+-----------------+------------------+-------------------+------------------+---------------+---------------+----------------+--------------------+----------------+---------------+-----+----------------+-------------------+--------------------+------------------+------------------+------------------+----------------+----------------+
 0| null| 165270|0.5817937658450281| 0.4019975433642836|ARD7TVE1187B99BFB1| null|California - LA| null|e77e51a5-4761-45b...| Casual| 4479| null| 0| 0| Fear Itself| 300848|0.6021199899057548|SOMZWCG12A8C13C480|I Didn't Mean To| 3401791|
 0| null| 1998|0.6306300375898077| 0.4174996449709784|ARMJAGH1187FB546F3| 35.14968| Memphis, TN| -90.04892|1c78ab62-db33-443...| The Box Tops| 22066| null| 0| 0| Dimensions| 300822| null|SOCIWDW12A8C13D406| Soul Deep| 3400270|
 0| null| 290021|0.4873567909281477|0.34342837829688244|ARKRRTF1187B9984DA| null| null| null|7a273984-edd9-445...|Sonora Santanera| 13951| null| 0| 0|Las Numero 1 De L...| 514953| null|SOXVLOJ12AB0189215| Amor De Cabaret| 5703798|
 0| null| 19072|0.6303823341467806| 0.4542311565706205|AR7G5I41187FB4CE6C| null|London, England| null|e188a520-9cb7-4f7...| Adam Ant| 12697| null| 0| 0| Friend Or Foe| 287650| null|SONHOTT12A8C13493C| Something Girls| 3226795|
 0| null| 30973|0.6510456608317947|0.40172368550367865|ARXR32B1187FB57099| null| null| null|c6903a2e-063c-4f9...| Gob| 8355| null| 0| 0| Muertos Vivos| 611336|0.6045007385888197|SOFSOCN12A8C143F5D| Face the Ashes| 6795666|
+---+----------------+-----------------+------------------+-------------------+------------------+---------------+---------------+----------------+--------------------+----------------+---------------+-----+----------------+-------------------+--------------------+------------------+------------------+------------------+----------------+----------------+
only showing top 5 rows

In [8]:
# change ids from strings to integers - this is needed for ALS
userId_change = raw_plays_df.select('userId').distinct()
songId_change = raw_plays_df.select('songId').distinct()
artistId_change = metadata_df.select('artist_id').distinct()

print('Number of unique users: {0}'.format(userId_change.count()))
print('Number of unique songs: {0}'.format(songId_change.count()))
print('Number of unique artists: {0}'.format(artistId_change.count()))

Number of unique users: 1019318
Number of unique songs: 384546
Number of unique artists: 3888

In [9]:
# change ids from strings to integers - this is needed for ALS
userId_change.createOrReplaceTempView('u')
userId_change = spark.sql('select row_number() over (order by "userId") as new_userId, * from u')

songId_change.createOrReplaceTempView('s')
songId_change = spark.sql('select row_number() over (order by "songId") as new_songId, * from s')

artistId_change.createOrReplaceTempView('a')
artistId_change = spark.sql('select row_number() over (order by "artist_id") as new_artistId, * from a')

In [10]:
userId_change.cache()
userId_change.show(5)

songId_change.cache()
songId_change.show(5)

artistId_change.cache()
artistId_change.show(5)

+----------+--------------------+
new_userId| userId|
+----------+--------------------+
 1|e0cfa84380bbf815c...|
 2|1db81ad6b41ced930...|
 3|39e032482d27268ed...|
 4|57d49f396c9c00002...|
 5|57b9feb2ca204c946...|
+----------+--------------------+
only showing top 5 rows

+----------+------------------+
new_songId| songId|
+----------+------------------+
 1|SOAMPRJ12A8AE45F38|
 2|SOIOXEH12AB0189AC7|
 3|SOBTTRJ12A8C141FB4|
 4|SOXKBTG12A6D4FA2AF|
 5|SOPVQLJ12A67AE2281|
+----------+------------------+
only showing top 5 rows

+------------+------------------+
new_artistId| artist_id|
+------------+------------------+
 1|AR0IAWL1187B9A96D0|
 2|AR1072G1187FB5B351|
 3|ARXZIHJ1187B9A4ED9|
 4|AR5R7791187FB3A8C3|
 5|AR5T4YC1187FB5915E|
+------------+------------------+
only showing top 5 rows

In [11]:
unique_users = userId_change.count()
unique_songs = songId_change.count()
unique_artists = artistId_change.count()
print('Number of unique users: {0}'.format(unique_users))
print('Number of unique songs: {0}'.format(unique_songs))
print('Number of unique artists: {0}'.format(unique_artists))

Number of unique users: 1019318
Number of unique songs: 384546
Number of unique artists: 3888

In [12]:
# join dataframes - append new Ids
raw_plays_df_with_int_ids = raw_plays_df.join(userId_change, 'userId').join(songId_change, 'songId')

# join dataframes - append original artist Ids and then new Ids
raw_plays_df_with_int_ids = raw_plays_df_with_int_ids.join(metadata_df.select('artist_id', 'songId'), 'songId')
raw_plays_df_with_int_ids = raw_plays_df_with_int_ids.join(artistId_change, 'artist_id')

# remove half users to make more manageable -- SKIP THIS STEP
#raw_plays_df_with_int_ids = raw_plays_df_with_int_ids.filter(raw_plays_df_with_int_ids.new_userId < unique_users/2)

print('Number of unique users: {0}'.format(unique_users))
print('Number of unique songs: {0}'.format(unique_songs))
print('Number of unique artists: {0}'.format(unique_artists))

# cache
raw_plays_df_with_int_ids.cache()
raw_plays_df_with_int_ids.show(5)

Number of unique users: 1019318
Number of unique songs: 384546
Number of unique artists: 3888
+------------------+------------------+--------------------+-----+----------+----------+------------+
 artist_id| songId| userId|Plays|new_userId|new_songId|new_artistId|
+------------------+------------------+--------------------+-----+----------+----------+------------+
ARXSABJ1187FB3C336|SOCKUUJ12A6D4FA41C|12be039b66971ca07...| 1| 815574| 305499| 626|
ARXSABJ1187FB3C336|SOCKUUJ12A6D4FA41C|4cb81273ceae78834...| 5| 816822| 305499| 626|
ARXSABJ1187FB3C336|SOCKUUJ12A6D4FA41C|5a4a3b4aeff30080e...| 5| 816806| 305499| 626|
ARXSABJ1187FB3C336|SOCKUUJ12A6D4FA41C|632dc6e049c989d1b...| 1| 818443| 305499| 626|
ARXSABJ1187FB3C336|SOCKUUJ12A6D4FA41C|921933941000d6cd9...| 1| 816207| 305499| 626|
+------------------+------------------+--------------------+-----+----------+----------+------------+
only showing top 5 rows

## Basic Recommendations

One way to recommend songs is to simply always recommend the songs with that are most listened to. In this part, We use Spark to find the name, total number of plays, filter those that have been listened to by more that 200 unique users to account for a broad range of appeal

### For Songs

In [15]:
song_ids_with_total_listens = raw_plays_df_with_int_ids.groupBy('songId') \
                                                       .agg(F.count(raw_plays_df_with_int_ids.Plays).alias('User_Count'),
                                                            F.sum(raw_plays_df_with_int_ids.Plays).alias('Total_Plays')) \
                                                       .orderBy('Total_Plays', ascending = False)

song_ids_with_total_listens.show(3, truncate=False)

+------------------+----------+-----------+
songId |User_Count|Total_Plays|
+------------------+----------+-----------+
SOULTKQ12AB018A183|30117 |87366 |
SOIZLKI12A6D4F7B61|28286 |75669 |
SOEHWGF12A6D4F8B2B|16128 |41301 |
+------------------+----------+-----------+
only showing top 3 rows

In [16]:
song_names_with_plays_df = song_ids_with_total_listens.join(metadata_df, 'songId' ) \
                                                      .filter('User_Count >= 200') \
                                                      .select('artist_name', 'title', 'songId', 'User_Count','Total_Plays') \
                                                      .orderBy('Total_Plays', ascending = False)

song_names_with_plays_df.show(3, truncate = False)

+-----------------------------+-------------------------------------------------+------------------+----------+-----------+
artist_name |title |songId |User_Count|Total_Plays|
+-----------------------------+-------------------------------------------------+------------------+----------+-----------+
B.o.B |Nothin' On You [feat. Bruno Mars] (Album Version)|SOULTKQ12AB018A183|30117 |87366 |
Muse |Supermassive Black Hole (Album Version) |SOIZLKI12A6D4F7B61|28286 |75669 |
Shakira Featuring Wyclef Jean|Hips Don't Lie (featuring Wyclef Jean) |SOEHWGF12A6D4F8B2B|16128 |41301 |
+-----------------------------+-------------------------------------------------+------------------+----------+-----------+
only showing top 3 rows

### For Artists

In [18]:
# Basic recommendation (popularity) -- ARTISTS

artist_ids_with_total_listens = raw_plays_df_with_int_ids.groupBy('artist_id') \
                                                       .agg(F.count(raw_plays_df_with_int_ids.Plays).alias('User_Count'),
                                                            F.sum(raw_plays_df_with_int_ids.Plays).alias('Total_Plays')) \
                                                       .orderBy('Total_Plays', ascending = False)
artist_ids_with_total_listens.show(5)

+------------------+----------+-----------+
 artist_id|User_Count|Total_Plays|
+------------------+----------+-----------+
ARR3ONV1187B9A2F59| 39983| 110308|
ARKQQZA12086C116FC| 30117| 87366|
ARQUMH41187B9AF699| 20659| 57866|
AR6PJ8R1187FB5AD70| 19858| 50212|
AR65K7A1187FB4DAA4| 20093| 46886|
+------------------+----------+-----------+
only showing top 5 rows

In [19]:
# Join with metadata to get artist details
artist_names_with_plays_df = artist_ids_with_total_listens.join(metadata_df, 'artist_id' ) \
                                                      .filter('User_Count >= 200') \
                                                      .select('artist_name', 'artist_id', 'User_Count','Total_Plays') \
                                                      .distinct() \
                                                      .orderBy('Total_Plays', ascending = False)

artist_names_with_plays_df.show(5)

+--------------------+------------------+----------+-----------+
 artist_name| artist_id|User_Count|Total_Plays|
+--------------------+------------------+----------+-----------+
 Muse|ARR3ONV1187B9A2F59| 39983| 110308|
 B.o.B|ARKQQZA12086C116FC| 30117| 87366|
 Linkin Park|ARQUMH41187B9AF699| 20659| 57866|
 Shakira|AR6PJ8R1187FB5AD70| 19858| 50212|
Shakira Featuring...|AR6PJ8R1187FB5AD70| 19858| 50212|
+--------------------+------------------+----------+-----------+
only showing top 5 rows

In [20]:
artist_ids_with_total_listens.show(3)

+------------------+----------+-----------+
 artist_id|User_Count|Total_Plays|
+------------------+----------+-----------+
ARR3ONV1187B9A2F59| 39983| 110308|
ARKQQZA12086C116FC| 30117| 87366|
ARQUMH41187B9AF699| 20659| 57866|
+------------------+----------+-----------+
only showing top 3 rows

## Collaborative Filtering

[Collaborative filtering][collab] is a method of making automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a person chosen randomly. 

----

For song recommendations, we start with a matrix whose entries are number of song plays by users.

Since not all users have listened to all songs, we do not know all of the entries in this matrix, which is precisely why we need collaborative filtering.  For each user, we have number of song plays for only a subset of the songs.  With collaborative filtering, the idea is to approximate the matrix by factorizing it as the product of two matrices: one that describes properties of each user, and one that describes properties of each song.


We want to select these two matrices such that the error for the users/songs pairs where we know the correct number of plays is minimized.  The Alternating Least Squares (ALS) algorithm does this by first randomly filling the users matrix with values and then optimizing the value of the songs such that the error is minimized.  Then, it holds the songs matrix constant and optimizes the value of the user's matrix.  This alternation between which matrix to optimize is the reason for the "alternating" in the name.

Given a fixed set of user factors (i.e., values in the users matrix), we use the known number of plays to find the best values for the songs factors using the least squares optimization.  Then we "alternate" and pick the best user factors given fixed songs factors.

In [22]:
# We'll hold out 60% for training, 20% of our data for validation, and leave 20% for testing
seed = 1
(split_60_df, split_a_20_df, split_b_20_df) = raw_plays_df_with_int_ids.randomSplit([0.6, 0.2, 0.2], seed = seed)

# Let's cache these datasets for performance
training_df = split_60_df.cache()
validation_df = split_a_20_df.cache()
test_df = split_b_20_df.cache()

print('Training: {0}, validation: {1}, test: {2}\n'.format(
  training_df.count(), validation_df.count(), test_df.count())
)
training_df.show(3)
validation_df.show(3)
test_df.show(3)


Training: 464130, validation: 154348, test: 154183

+------------------+------------------+--------------------+-----+----------+----------+------------+
 artist_id| songId| userId|Plays|new_userId|new_songId|new_artistId|
+------------------+------------------+--------------------+-----+----------+----------+------------+
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|03699fa50d944261d...| 1| 427565| 349| 1831|
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|15ae83218a98bdabc...| 1| 208894| 349| 1831|
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|193b52a23322f3c38...| 1| 683290| 349| 1831|
+------------------+------------------+--------------------+-----+----------+----------+------------+
only showing top 3 rows

+------------------+------------------+--------------------+-----+----------+----------+------------+
 artist_id| songId| userId|Plays|new_userId|new_songId|new_artistId|
+------------------+------------------+--------------------+-----+----------+----------+------------+
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|3888b133d6bcd161f...| 3| 534451| 349| 1831|
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|3d1d9a7e246b95467...| 7| 532304| 349| 1831|
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|95e728c940b51dc1e...| 5| 229691| 349| 1831|
+------------------+------------------+--------------------+-----+----------+----------+------------+
only showing top 3 rows

+------------------+------------------+--------------------+-----+----------+----------+------------+
 artist_id| songId| userId|Plays|new_userId|new_songId|new_artistId|
+------------------+------------------+--------------------+-----+----------+----------+------------+
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|2dbe9b45bb2362b87...| 1| 291708| 349| 1831|
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|56dd58c313226feea...| 1| 1004763| 349| 1831|
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|a95131cfe02735ab9...| 1| 726012| 349| 1831|
+------------------+------------------+--------------------+-----+----------+----------+------------+
only showing top 3 rows

In [23]:
#Number of plays needs to be double type, not integers
validation_df = validation_df.withColumn("Plays", validation_df["Plays"].cast(DoubleType()))
validation_df.show(10)

+------------------+------------------+--------------------+-----+----------+----------+------------+
 artist_id| songId| userId|Plays|new_userId|new_songId|new_artistId|
+------------------+------------------+--------------------+-----+----------+----------+------------+
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|3888b133d6bcd161f...| 3.0| 720266| 1089| 3841|
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|3d1d9a7e246b95467...| 7.0| 723516| 1089| 3841|
AR1PCLK1187FB5BBFF|SOJUGKQ12A8C13A83A|95e728c940b51dc1e...| 5.0| 209106| 1089| 3841|
AR77MEJ1187B98F8CA|SOEXTFN12AB0188B18|6a21c2837209d3cf1...| 2.0| 65884| 1758| 3134|
ARABTW91187FB3A73B|SOJUBCK12A81C2050B|0253abf0972ac4efe...| 1.0| 515690| 1078| 1816|
ARABTW91187FB3A73B|SOJUBCK12A81C2050B|627897f8fa044dbb0...| 1.0| 65585| 1078| 1816|
ARABTW91187FB3A73B|SOJUBCK12A81C2050B|8f7f0e310936aa71b...| 1.0| 933993| 1078| 1816|
ARABTW91187FB3A73B|SOJUBCK12A81C2050B|f7b397d85f435e226...| 5.0| 430590| 1078| 1816|
ARAIJG51187B9B97A4|SOTSIIH12A8C13A516|002abc31db3d2d789...| 1.0| 695282| 106| 2142|
ARAIJG51187B9B97A4|SOTSIIH12A8C13A516|003c6a97c6f53c5ee...| 1.0| 227966| 106| 2142|
+------------------+------------------+--------------------+-----+----------+----------+------------+
only showing top 10 rows

In [24]:
artist_plays_df = raw_plays_df_with_int_ids.groupBy("artist_id", "userId", "new_userId", "new_artistId").agg(F.sum(raw_plays_df_with_int_ids.Plays).alias('Plays'))

artist_plays_df.show(5)

+------------------+--------------------+----------+------------+-----+
 artist_id| userId|new_userId|new_artistId|Plays|
+------------------+--------------------+----------+------------+-----+
ARG9CID1187B9AB010|0c91fa7d9e220c56b...| 631103| 1114| 1|
ARG9CID1187B9AB010|bd899d5f2cadff3ce...| 10794| 1114| 1|
ARG9CID1187B9AB010|5ab8603f502511e31...| 248786| 1114| 2|
ARG9CID1187B9AB010|9a14c2e509eab28dd...| 378990| 1114| 1|
ARF3HEO1187B9AF2F4|3cbb10ce899144d04...| 126120| 1192| 3|
+------------------+--------------------+----------+------------+-----+
only showing top 5 rows

In [25]:
# create test, trian, validation for Artist

# We'll hold out 60% for training, 20% of our data for validation, and leave 20% for testing
seed = 1
(split_60_artist, split_a_20_artist, split_b_20_artist) = artist_plays_df.randomSplit([0.6, 0.2, 0.2], seed = seed)

# Let's cache these datasets for performance
training_artist = split_60_artist.cache()
validation_artist = split_a_20_artist.cache()
test_artist = split_b_20_artist.cache()

print('Training: {0}, validation: {1}, test: {2}\n'.format(
  training_artist.count(), validation_artist.count(), test_artist.count())
)

#Number of plays needs to be double type, not integers
training_artist = training_artist.withColumn("Plays", training_artist["Plays"].cast(DoubleType()))
validation_artist = validation_artist.withColumn("Plays", validation_artist["Plays"].cast(DoubleType()))
test_artist = test_artist.withColumn("Plays", test_artist["Plays"].cast(DoubleType()))

training_artist.show(3)
validation_artist.show(3)
test_artist.show(3)

Training: 425101, validation: 141542, test: 141972

+------------------+--------------------+----------+------------+-----+
 artist_id| userId|new_userId|new_artistId|Plays|
+------------------+--------------------+----------+------------+-----+
AR02YGA1187B9B8AC4|233c25118865ca3ac...| 25015| 2472| 2.0|
AR02YGA1187B9B8AC4|48f561ec4070412fe...| 673630| 2472| 1.0|
AR02YGA1187B9B8AC4|4ced3be5f889df34e...| 910705| 2472| 2.0|
+------------------+--------------------+----------+------------+-----+
only showing top 3 rows

+------------------+--------------------+----------+------------+-----+
 artist_id| userId|new_userId|new_artistId|Plays|
+------------------+--------------------+----------+------------+-----+
AR03BDP1187FB5B324|0d87331d41e1e276c...| 607859| 3557| 16.0|
AR03BDP1187FB5B324|1e4ad0a7ca33b49b1...| 698799| 3557| 1.0|
AR03BDP1187FB5B324|8ea9dd54010b157e0...| 304805| 3557| 1.0|
+------------------+--------------------+----------+------------+-----+
only showing top 3 rows

+------------------+--------------------+----------+------------+-----+
 artist_id| userId|new_userId|new_artistId|Plays|
+------------------+--------------------+----------+------------+-----+
AR035N21187FB3938E|c3644f56bd6a8b343...| 663988| 107| 1.0|
AR03BDP1187FB5B324|3bc20a78491a1395c...| 276154| 3557| 4.0|
AR03BDP1187FB5B324|b72dd8684a6b54761...| 1003835| 3557| 1.0|
+------------------+--------------------+----------+------------+-----+
only showing top 3 rows

### Alternating Least Squares

In this part, we will use the Apache Spark ML Pipeline implementation of Alternating Least Squares, [ALS](http://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.recommendation.ALS). ALS takes a training dataset (DataFrame) and several parameters that control the model creation process. To determine the best values for the parameters, we will use ALS to train several models, and then we will select the best model and use the parameters from that model in the rest of the analysis.

The process we will use for determining the best model is as follows:
1. Pick a set of model parameters. The most important parameter to model is the *rank*, which is the number of columns in the Users matrix or the number of rows in the Movies matrix (blue in the diagram above). In general, a lower rank will mean higher error on the training dataset, but a high rank may lead to [overfitting](https://en.wikipedia.org/wiki/Overfitting).  We will train models with ranks of 4, 8, 12 and 16 using the `training_df` dataset, as well as iterating over the regularization parameter. Because most of the number of songs are have only be listened to once, I expect a higher regularization parameter to be more effective.

2. We set the appropriate parameters on the `ALS` object:
    * The "User" column will be set to the values in our `new_userId` DataFrame column.
    * The "Item" column will be set to the values in our `new_songId` or `new_artistId` DataFrame column.
    * The "Rating" column will be set to the values in our `Plays` DataFrame column.
    * We'll set the max number of iterations to be 5.

3. Have the ALS output transformation (i.e., the result of [ALS.fit()](http://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.recommendation.ALS.fit)) produce a _new_ column
   called "prediction" that contains the predicted value.

4. Create multiple models using [ALS.fit()](http://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.recommendation.ALS.fit), one for each of our rank values. We'll fit
   against the training data set (`training_df`).

5. For each model, we'll run a prediction against our validation data set (`validation_df`) and check the error.

6. We'll keep the model with the best error rate.

In [27]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Let's initialize our ALS learner
als = ALS()

# Now set the parameters for the method
als.setMaxIter(5)\
   .setSeed(seed)\
   .setItemCol("new_songId")\
   .setRatingCol("Plays")\
   .setUserCol("new_userId")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="Plays", metricName="rmse")

tolerance = 0.03
ranks = [4, 8, 12, 16]
regParams = [0.15, 0.2, 0.25]
errors = [[0]*len(ranks)]*len(regParams)
models = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1
i = 0
for regParam in regParams:
  j = 0
  for rank in ranks:
    # Set the rank here:
    als.setParams(rank = rank, regParam = regParam)
    # Create the model with these parameters.
    model = als.fit(training_df)
    # Run the model to create a prediction. Predict against the validation_df.
    predict_df = model.transform(validation_df)

    # Remove NaN values from prediction (due to SPARK-14489)
    predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
    predicted_plays_df = predicted_plays_df.withColumn("prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))
    # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
    error = reg_eval.evaluate(predicted_plays_df)
    errors[i][j] = error
    models[i][j] = model
    print ('For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error))
    if error < min_error:
      min_error = error
      best_params = [i,j]
      j += 1
  i += 1

als.setRegParam(regParams[best_params[0]])
als.setRank(ranks[best_params[1]])
print ('The best model was trained with regularization parameter %s' % regParams[best_params[0]])
print ('The best model was trained with rank %s' % ranks[best_params[1]])
my_model = models[best_params[0]][best_params[1]]

For rank 4, regularization parameter 0.15 the RMSE is 5.305359899900203
For rank 8, regularization parameter 0.15 the RMSE is 5.057682557275098
For rank 12, regularization parameter 0.15 the RMSE is 5.037615304177453
For rank 16, regularization parameter 0.15 the RMSE is 4.995392506309996
For rank 4, regularization parameter 0.2 the RMSE is 5.241601352929219
For rank 8, regularization parameter 0.2 the RMSE is 5.030263426872916
For rank 12, regularization parameter 0.2 the RMSE is 5.011767767875298
For rank 16, regularization parameter 0.2 the RMSE is 4.9739499486300005
For rank 4, regularization parameter 0.25 the RMSE is 5.1978961767270295
For rank 8, regularization parameter 0.25 the RMSE is 5.020512065791947
For rank 12, regularization parameter 0.25 the RMSE is 4.994552621921719
For rank 16, regularization parameter 0.25 the RMSE is 4.963960846433318
The best model was trained with regularization parameter 0.25
The best model was trained with rank 4

In [28]:
# Let's initialize our ALS learner
als = ALS()

# Now set the parameters for the method
als.setMaxIter(5)\
   .setSeed(seed)\
   .setItemCol("new_songId")\
   .setRatingCol("Plays")\
   .setUserCol("new_userId")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="Plays", metricName="rmse")

als.setParams(rank = 4, regParam = 0.25)
model = als.fit(training_df)
predict_df = model.transform(validation_df)
predict_test = model.transform(test_df)

In [29]:
# Remove NaN values from prediction (due to SPARK-14489)
predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
predicted_plays_df = predicted_plays_df.withColumn("prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))

In [30]:
#Example of predicted plays
predicted_plays_df.show(10)

+------------------+------------------+--------------------+-----+----------+----------+------------+----------+
 artist_id| songId| userId|Plays|new_userId|new_songId|new_artistId|prediction|
+------------------+------------------+--------------------+-----+----------+----------+------------+----------+
ART62WC1187FB47E80|SOXMSNK12AF72A8850|e8a8acbdd37148eac...| 4| 242656| 73470| 1223| 1.0|
ARQ3VBJ1187FB3FE55|SOWXUDU12AF72A1001|bb54c33e5a18870c6...| 1| 336979| 102960| 879| 0.0|
ARLUF1T1187FB3603B|SOPHJIS12AC3DF5FA3|b8306bb90437dcc23...| 1| 604887| 156645| 3657| 0.0|
ARLUF1T1187FB3603B|SOPHJIS12AC3DF5FA3|43a3d80ef57b997c0...| 1| 754737| 156645| 3657| 0.0|
ARLUF1T1187FB3603B|SOPHJIS12AC3DF5FA3|ea2131622f74b05cd...| 2| 460249| 156645| 3657| 0.0|
ARLUF1T1187FB3603B|SOPHJIS12AC3DF5FA3|8b732d17887245910...| 2| 242138| 156645| 3657| 0.0|
ARLUF1T1187FB3603B|SOPHJIS12AC3DF5FA3|0987a9043d13b64f0...| 1| 512616| 156645| 3657| 1.0|
ARV1P811187FB3CFC6|SOGMXBW12A6D4FB8D7|6be9cb80937e3fdb2...| 1| 532621| 180981| 2| 2.0|
ARV1P811187FB3CFC6|SOGMXBW12A6D4FB8D7|6bf609c8a472dd82b...| 1| 849748| 180981| 2| 2.0|
ARV1P811187FB3CFC6|SOGMXBW12A6D4FB8D7|e02791cf50396bef5...| 5| 762445| 180981| 2| 15.0|
+------------------+------------------+--------------------+-----+----------+----------+------------+----------+
only showing top 10 rows

Because we're only looking for the nonzero plays, one way to increase the accuracy would be to convert predicted listens that are zero to one. However this would not really help the model overall, since our ultimate goal is for song recommendation and if the predicted number of plays is zero that is okay, this means that the user does o want to listen to the song.

### Testing The Model

So far, we used the `training_df` and `validation_df` datasets to select the best model.  Since we used these two datasets to determine what model is best, we cannot use them to test how good the model is; otherwise, we would be very vulnerable to [overfitting](https://en.wikipedia.org/wiki/Overfitting).  To decide how good the model is, we need to use the `test_df` dataset.  We will use the best rank, and best regularization parameter in the list `best_params` previously determined to create a model for predicting songs for the test dataset and then we will compute the RMSE.

In [33]:
# In ML Pipelines, this next step has a bug that produces unwanted NaN values. We
# have to filter them out. See https://issues.apache.org/jira/browse/SPARK-14489

test_df = test_df.withColumn("Plays", test_df["Plays"].cast(DoubleType()))
predict_test_df = my_model.transform(test_df)

# Remove NaN values from prediction (due to SPARK-14489)
predicted_test_df = predict_df.filter(predict_df.prediction != float('nan'))

# Round floats to whole numbers
predicted_test_df = predicted_test_df.withColumn("prediction", F.abs(F.round(predicted_test_df["prediction"],0)))
# Run the previously created RMSE evaluator, reg_eval, on the predicted_test_df DataFrame
test_RMSE = reg_eval.evaluate(predicted_test_df)

print('The model had a RMSE on the test set of {0}'.format(test_RMSE))

The model had a RMSE on the test set of 9.829685579527544

### Comparing the Model

Looking at the RMSE for the results predicted by the model versus the values in the test set is one way to evalute the quality of our model. Another way to evaluate the model is to evaluate the error from a test set where every rating is the average number of plays from the training set.

In [35]:
avg_plays_df = training_df.groupBy().avg('Plays').select(F.round('avg(Plays)'))

avg_plays_df.show(3)
# Extract the average rating value. (This is row 0, column 0.)
training_avg_plays = avg_plays_df.collect()[0][0]

print('The average number of plays in the dataset is {0}'.format(training_avg_plays))

# Add a column with the average rating
test_for_avg_df = test_df.withColumn('prediction', F.lit(training_avg_plays))

# Run the previously created RMSE evaluator, reg_eval, on the test_for_avg_df DataFrame
test_avg_RMSE = reg_eval.evaluate(test_for_avg_df)

print("The RMSE on the average set is {0}".format(test_avg_RMSE))

+--------------------+
round(avg(Plays), 0)|
+--------------------+
 3.0|
+--------------------+

The average number of plays in the dataset is 3.0
The RMSE on the average set is 5.524487897830476

Our model performs slightly better than simply predicting 3 songs for all in the test case.

## Prediction for a User

We can make a prediction for a user. To do this we make a list of all the songs that the user has not already listened to, and feed it into to model. The model will generate it the number of predicted plays, which we can arange in descending order.

In [38]:
spark.conf.set("spark.sql.crossJoin.enabled", "true")

In [39]:
# actual listens
raw_plays_df_with_int_ids.filter(raw_plays_df_with_int_ids.new_userId == 3543).join(metadata_df, 'songId').select('artist_name', 'title'). distinct().orderBy('Plays', ascending=False).show()

+--------------------+--------------------+
 artist_name| title|
+--------------------+--------------------+
Bob Marley & The ...| Smile Jamaica|
 Muse|Supermassive Blac...|
 The Rolling Stones|Angie (1993 Digit...|
 Hot Chip| Hold On|
 The Killers|A Dustland Fairytale|
 Aerosmith| Crazy|
+--------------------+--------------------+

In [40]:
UserID = 3543
listened_songs = raw_plays_df_with_int_ids.filter(raw_plays_df_with_int_ids.new_userId == UserID) \
                                          .join(metadata_df, 'songId') \
                                          .select('new_songId', 'artist_name', 'title') \
                                          
# generate list of listened songs
listened_songs_list = []
for song in listened_songs.collect():
  listened_songs_list.append(song['new_songId'])

print('Songs user has listened to:')
listened_songs.select('artist_name', 'title').show()

# generate dataframe of unlistened songs
unlistened_songs = raw_plays_df_with_int_ids.filter(~ raw_plays_df_with_int_ids['new_songId'].isin(listened_songs_list)) \
                                            .select('new_songId').withColumn('new_userId', F.lit(UserID)).distinct()

# feed unlistened songs into model
predicted_listens = model.transform(unlistened_songs)

# remove NaNs
predicted_listens = predicted_listens.filter(predicted_listens['prediction'] != float('nan'))

# print output
print('Predicted Songs:')
predicted_listens.join(raw_plays_df_with_int_ids, 'new_songId') \
                 .join(metadata_df, 'songId') \
                 .select('artist_name', 'title', 'prediction') \
                 .distinct() \
                 .orderBy('prediction', ascending = False) \
                 .show(10)

Songs user has listened to:
+------------------+--------------------+
 artist_name| title|
+------------------+--------------------+
 The Killers|A Dustland Fairytale|
The Rolling Stones|Angie (1993 Digit...|
 Hot Chip| Hold On|
+------------------+--------------------+

Predicted Songs:
+--------------------+--------------------+----------+
 artist_name| title|prediction|
+--------------------+--------------------+----------+
 Memphis La Blusera|Exactamente Media...| 10.494117|
 UFO|Out In The Street...| 5.915309|
 Less Than Jake|Antidote For The ...| 5.8477798|
 Bobaflex| Medicine| 5.2904453|
 Michael McDonald| Nowhere To Run| 4.9717727|
 Janet Paschal| God Will Make A Way| 4.4560666|
 Hem|Have Yourself A M...| 4.2125645|
Joe Graves & The ...|Witchcraft (1997 ...| 4.2047486|
 The Scare| I Saw Destruction| 4.1932917|
 Green Day|Wake Me Up When S...| 3.5484014|
+--------------------+--------------------+----------+
only showing top 10 rows

### ALS for Artists

In [42]:
# Let's initialize our ALS learner
als_artist = ALS()

# Now set the parameters for the method
als_artist.setMaxIter(5)\
   .setSeed(seed)\
   .setItemCol("new_artistId")\
   .setRatingCol("Plays")\
   .setUserCol("new_userId")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="Plays", metricName="rmse")

tolerance = 0.03
ranks = [4, 8, 12, 16]
regParams = [0.15, 0.2, 0.25]
errors_artist = [[0]*len(ranks)]*len(regParams)
models_artist = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1
i = 0
for regParam in regParams:
  j = 0
  for rank in ranks:
    # Set the rank here:
    als_artist.setParams(rank = rank, regParam = regParam)
    # Create the model with these parameters.
    model_artist = als_artist.fit(training_artist)
    # Run the model to create a prediction. Predict against the validation_df.
    predict_artist = model_artist.transform(validation_artist)

    # Remove NaN values from prediction (due to SPARK-14489)
    predicted_plays_artist = predict_artist.filter(predict_artist.prediction != float('nan'))
    predicted_plays_artist = predicted_plays_artist.withColumn("prediction", F.abs(F.round(predicted_plays_artist["prediction"],0)))
    # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
    error_artist = reg_eval.evaluate(predicted_plays_artist)
    errors_artist[i][j] = error_artist
    models_artist[i][j] = model_artist
    print ('For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error_artist))
    if error_artist < min_error:
      min_error = error_artist
      best_params = [i,j]
      j += 1
  i += 1

als_artist.setRegParam(regParams[best_params[0]])
als_artist.setRank(ranks[best_params[1]])
print ('The best model was trained with regularization parameter %s' % regParams[best_params[0]])
print ('The best model was trained with rank %s' % ranks[best_params[1]])
my_model_artist = models_artist[best_params[0]][best_params[1]]

For rank 4, regularization parameter 0.15 the RMSE is 5.734077983767863
For rank 8, regularization parameter 0.15 the RMSE is 5.602468800547022
For rank 12, regularization parameter 0.15 the RMSE is 5.534962572321722
For rank 16, regularization parameter 0.15 the RMSE is 5.521896682401342
For rank 4, regularization parameter 0.2 the RMSE is 5.735768803402946
For rank 8, regularization parameter 0.2 the RMSE is 5.580275472972566
For rank 12, regularization parameter 0.2 the RMSE is 5.529227259704196
For rank 16, regularization parameter 0.2 the RMSE is 5.513880411199703
For rank 4, regularization parameter 0.25 the RMSE is 5.72640444994007
For rank 8, regularization parameter 0.25 the RMSE is 5.569391380543844
For rank 12, regularization parameter 0.25 the RMSE is 5.530099574273815
For rank 16, regularization parameter 0.25 the RMSE is 5.515329688460216
The best model was trained with regularization parameter 0.2
The best model was trained with rank 4

In [43]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Let's initialize our ALS learner
als_artist = ALS()

# Now set the parameters for the method
als_artist.setMaxIter(5)\
   .setSeed(seed)\
   .setItemCol("new_artistId")\
   .setRatingCol("Plays")\
   .setUserCol("new_userId")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="Plays", metricName="rmse")

als_artist.setParams(rank = 4, regParam = 0.2)
model_artist = als_artist.fit(training_artist)
predict_artist = model_artist.transform(validation_artist)
predict_artist_test = model_artist.transform(test_artist)

In [44]:
# Remove NaN values from prediction
predicted_plays_artist = predict_artist.filter(predict_artist.prediction != float('nan'))
predicted_plays_artist = predicted_plays_artist.withColumn("prediction", F.abs(F.round(predicted_plays_artist["prediction"],0)))

# Remove NaN values from prediction
predicted_plays_artist_test = predict_artist_test.filter(predict_artist_test.prediction != float('nan'))
predicted_plays_artist_test = predicted_plays_artist_test.withColumn("prediction", F.abs(F.round(predicted_plays_artist_test["prediction"],0)))

predicted_plays_artist.show(10)
predicted_plays_artist_test.show(10)

+------------------+--------------------+----------+------------+-----+----------+
 artist_id| userId|new_userId|new_artistId|Plays|prediction|
+------------------+--------------------+----------+------------+-----+----------+
ARV2X851187FB41A78|7e6472406a6c34a91...| 1006689| 471| 1.0| 1.0|
ARV2X851187FB41A78|af4fec6bed2ef2b79...| 935232| 471| 1.0| 0.0|
ARVQTZP1187FB40283|b80c971530f3647e3...| 549849| 496| 4.0| 4.0|
ARVQTZP1187FB40283|c9593bfa9b57bcc77...| 268227| 496| 1.0| 0.0|
ARVQTZP1187FB40283|3f0489c87e1c70fb6...| 301174| 496| 1.0| 0.0|
ARDE7I31187FB43352|1dd53958710d04e16...| 177416| 1342| 1.0| 1.0|
ARDE7I31187FB43352|c5132693103017c69...| 33264| 1342| 2.0| 0.0|
ARDE7I31187FB43352|1fcb08f07e74b5d3b...| 617467| 1342| 3.0| 0.0|
ARP07GD1187B9B9DA4|aeb513f74b485e939...| 505901| 1591| 29.0| 2.0|
ARP07GD1187B9B9DA4|bcd6eac0cd2deaf5d...| 585301| 1591| 5.0| 1.0|
+------------------+--------------------+----------+------------+-----+----------+
only showing top 10 rows

+------------------+--------------------+----------+------------+-----+----------+
 artist_id| userId|new_userId|new_artistId|Plays|prediction|
+------------------+--------------------+----------+------------+-----+----------+
ARV2X851187FB41A78|831d30d970f636d9e...| 398466| 471| 1.0| 4.0|
ARV2X851187FB41A78|f63ad5ddd1208aedc...| 845670| 471| 14.0| 3.0|
ARVQTZP1187FB40283|6fc243a84a5bb98d3...| 310898| 496| 1.0| 1.0|
ARVQTZP1187FB40283|0bd2057b39d8f25d5...| 394339| 496| 2.0| 11.0|
ARVQTZP1187FB40283|179aff4cf2ef87d5d...| 107061| 496| 6.0| 4.0|
ARDE7I31187FB43352|d7258a427664641de...| 243304| 1342| 1.0| 0.0|
ARDE7I31187FB43352|b36d3f2ded02ec376...| 10115| 1342| 1.0| 2.0|
ARDE7I31187FB43352|36e70d2994e7bf147...| 508017| 1342| 1.0| 3.0|
ARP07GD1187B9B9DA4|244bf0abc538b39be...| 622727| 1591| 1.0| 2.0|
AROW3XO1187B989E72|1340dba813819075b...| 261655| 1829| 1.0| 1.0|
+------------------+--------------------+----------+------------+-----+----------+
only showing top 10 rows

### Acutal vs Prediction for a User

In [46]:
# actual
artist_plays_df.filter(artist_plays_df.new_userId == 3543).join(metadata_df, 'artist_id').select('artist_name', 'Plays').distinct().orderBy('Plays', ascending=False).show(10)

+--------------------+-----+
 artist_name|Plays|
+--------------------+-----+
 Bob Marley| 9|
Bob Marley & The ...| 9|
 Hot Chip| 1|
 Aerosmith| 1|
 The Killers| 1|
 Muse| 1|
 Killers| 1|
 The Rolling Stones| 1|
+--------------------+-----+

In [47]:
artist_plays_df.filter(artist_plays_df.artist_id == 'ARAZCWP129462A97F1').orderBy('Plays', ascending=False).show()

+------------------+--------------------+----------+------------+-----+
 artist_id| userId|new_userId|new_artistId|Plays|
+------------------+--------------------+----------+------------+-----+
ARAZCWP129462A97F1|ae304ac87c46c43cc...| 474241| 903| 81|
ARAZCWP129462A97F1|3dd582756ab8a9919...| 658141| 903| 73|
ARAZCWP129462A97F1|b2c5cff69924d56da...| 780942| 903| 72|
ARAZCWP129462A97F1|d8b14e66e9ddffa24...| 336068| 903| 70|
ARAZCWP129462A97F1|178d21a236c197a37...| 363018| 903| 57|
ARAZCWP129462A97F1|5cdd87d64c988b636...| 488001| 903| 52|
ARAZCWP129462A97F1|e4cca207d304cc3bb...| 623138| 903| 44|
ARAZCWP129462A97F1|aa3f9b412b658843b...| 13694| 903| 44|
ARAZCWP129462A97F1|290820cba8d8b2a82...| 691543| 903| 43|
ARAZCWP129462A97F1|44bc09d1c85903939...| 962387| 903| 40|
ARAZCWP129462A97F1|6d09b464b8dafe612...| 101795| 903| 40|
ARAZCWP129462A97F1|9e809d6b70bafd3bf...| 748005| 903| 40|
ARAZCWP129462A97F1|e7a4aff181c9fb8d0...| 963015| 903| 37|
ARAZCWP129462A97F1|d85008f99692044c7...| 812234| 903| 36|
ARAZCWP129462A97F1|53de39c4ff2e3abc6...| 403638| 903| 36|
ARAZCWP129462A97F1|0c5103399334f0baa...| 50315| 903| 35|
ARAZCWP129462A97F1|9f7da0eddcd0eb1c3...| 980727| 903| 35|
ARAZCWP129462A97F1|8e0481b4cbb9c636e...| 404390| 903| 34|
ARAZCWP129462A97F1|828a8e55f58c1e611...| 746274| 903| 34|
ARAZCWP129462A97F1|b660e7b6264046ca9...| 665601| 903| 33|
+------------------+--------------------+----------+------------+-----+
only showing top 20 rows

In [48]:
artist_plays_df.filter(artist_plays_df.new_userId == 780942).join(metadata_df, 'artist_id').select('artist_name', 'new_artistId', 'Plays').distinct().show()

+--------------------+------------+-----+
 artist_name|new_artistId|Plays|
+--------------------+------------+-----+
 Bob Marley| 903| 72|
Jack Johnson and ...| 2214| 23|
Bob Marley & The ...| 903| 72|
 Jack Johnson| 2214| 23|
+--------------------+------------+-----+

In [49]:
artist_plays_df.filter((artist_plays_df.new_artistId == 3784) | (artist_plays_df.new_artistId == 453) | (artist_plays_df.new_artistId == 88) | (artist_plays_df.new_artistId == 2214)).join(metadata_df, 'artist_id').sort('Plays', ascending=False).select('artist_name').distinct().show()

+--------------------+
 artist_name|
+--------------------+
 Sean Paul|
 Jack Johnson|
Jack Johnson and ...|
Beyoncé feat. Sli...|
 Misfits|
 Radiohead|
 Beyoncé|
 The Misfits|
+--------------------+

In [50]:
UserID = 980727
listened_artists = artist_plays_df.filter(artist_plays_df.new_userId == UserID) \
                                          .join(metadata_df, 'artist_id') \
                                          .select('new_artistId', 'artist_name', 'title') \
                                          
# generate list of listened artists
listened_artists_list = []
for song in listened_artists.collect():
  listened_artists_list.append(song['new_artistId'])

print('Artists user has listened to:')
listened_artists.select('artist_name').distinct().show()

# generate dataframe of unlistened songs
unlistened_artists = artist_plays_df.filter(~ artist_plays_df['new_artistId'].isin(listened_artists_list)) \
                                            .select('new_artistId').withColumn('new_userId', F.lit(UserID)).distinct()

# feed unlistened songs into model
predicted_artists = model_artist.transform(unlistened_artists)

# remove NaNs
predicted_artists = predicted_artists.filter(predicted_artists['prediction'] != float('nan'))

# print output
print('Predicted Songs:')
predicted_artists.join(artist_plays_df, 'new_artistId') \
                 .join(metadata_df, 'artist_id') \
                 .select('artist_name', 'title', 'prediction') \
                 .distinct() \
                 .orderBy('prediction', ascending = False) \
                 .show(10)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1570243133171221> in <module> () 
 20 # print output 
 21 print ( 'Predicted Songs:' ) 
 ---> 22 predicted_artists . join ( artist_plays_df , 'new_artistId' ) . join ( metadata_df , 'artist_id' ) . select ( 'artist_name' , 'title' , 'prediction' ) . distinct ( ) . orderBy ( 'prediction' , ascending = False ) . show ( 10 ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 377 """
 378 if isinstance ( truncate , bool ) and truncate : 
 --> 379 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 380 else : 
 381 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o2738.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 2849.0 failed 4 times, most recent failure: Lost task 5.3 in stage 2849.0 (TID 26868, 10.39.231.53, executor 4): java.lang.ArrayIndexOutOfBoundsException

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2355)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2343)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2342)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2342)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1096)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1096)
	at scala.Option.foreach(Option.scala:257)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskSetFailed(DAGScheduler.scala:1096)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2574)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2522)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2510)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:893)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2243)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2341)
	at org.apache.spark.rdd.RDD$$anonfun$takeOrdered$1.apply(RDD.scala:1467)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:379)
	at org.apache.spark.rdd.RDD.takeOrdered(RDD.scala:1442)
	at org.apache.spark.sql.execution.TakeOrderedAndProjectExec.executeCollect(limit.scala:154)
	at org.apache.spark.sql.execution.collect.Collector$.collect(Collector.scala:67)
	at org.apache.spark.sql.execution.collect.Collector$.colle

In [51]:
listened_songs.select('artist_name', 'title').show()
unlistened_songs = raw_plays_df_with_int_ids.filter(~ raw_plays_df_with_int_ids['new_songId'].isin(listened_songs_list)) \
                                            .select('new_songId').withColumn('new_userId', F.lit(UserID)).distinct()

predicted_listens_2 = model_2.transform(unlistened_songs)
predicted_listens_2 = predicted_listens_2.filter(predicted_listens_2['prediction'] != float('nan'))

print('Predicted Songs:')
predicted_listens_2.join(raw_plays_df_with_int_ids, 'new_songId') \
                 .join(metadata_df, 'songId') \
                 .select('artist_name', 'title', 'prediction') \
                 .distinct() \
                 .orderBy('prediction', ascending = False) \
                 .show(10)

In [52]:
## Testing
# In ML Pipelines, this next step has a bug that produces unwanted NaN values

predicted_test_artist = my_model_artist.transform(test_artist)

# Remove NaN values from prediction (due to SPARK-14489)
predicted_test_artist = predicted_test_artist.filter(predicted_test_artist.prediction != float('nan'))

# Round floats to whole numbers
predicted_test_artist = predicted_test_artist.withColumn("prediction", F.abs(F.round(predicted_test_artist["prediction"],0)))
# Run the previously created RMSE evaluator, reg_eval, on the predicted_test_df DataFrame
test_artist_RMSE = reg_eval.evaluate(predicted_test_artist)

print('The model had a RMSE on the test set of {0}'.format(test_artist_RMSE))

The model had a RMSE on the test set of 6.716275649987533

<br>

## Making Predictions Based on Songs Listened to at Least Twice

In order to lower the error I hypothesize that if a user has only listened to a song once, it is not representative of their listening profile. Listening to songs at least twice may be more representative and yield lower errors in the model predictions. Also since we want to recommend songs that the user likes, and will hopefully listen to more than once, it makes sense to use train on songs that have been listened to more than once.

In [54]:
raw_plays_df_2more_plays = raw_plays_df.join(userId_change, 'userId') \
                                       .join(songId_change, 'songId') \
                                       .filter(raw_plays_df.Plays >= 2)\
                                       .distinct()

tot_entries_2more = raw_plays_df_2more_plays.count()
print('Total enties with two or more plays: {0}'.format(tot_entries_2more))

raw_plays_df_2more_plays = raw_plays_df_2more_plays.filter(raw_plays_df_2more_plays.new_userId < (userId_change.count())*0.8) \
                                                   .select('new_userId', 'new_songId', 'Plays')
raw_plays_df_2more_plays.cache() 

Total enties with two or more plays: 19617620
 Out[ 53 ]: DataFrame[new_userId: int, new_songId: int, Plays: int]

We still see one number (2) which dominates, at around 37%, however this is much better than for one plays, which had almost 60%

Split into training, validation, and test datasets

In [57]:
# We'll hold out 60% for training, 20% of our data for validation, and leave 20% for testing
#seed = 1800083193L
seed = 1
(split_60_df_2more, split_a_20_df_2more, split_b_20_df_2more) = raw_plays_df_2more_plays.randomSplit([0.6, 0.2, 0.2], seed = seed)

# Let's cache these datasets for performance
training_2more_df = split_60_df_2more.cache()
validation_2more_df = split_a_20_df_2more.cache()
test_2more_df = split_b_20_df_2more.cache()

print('Training: {0}, validation: {1}, test: {2}\n'.format(
  training_2more_df.count(), validation_2more_df.count(), test_2more_df.count())
)
validation_2more_df = validation_2more_df.withColumn("Plays", validation_2more_df["Plays"].cast(DoubleType()))
test_2more_df = test_2more_df.withColumn("Plays", test_2more_df["Plays"].cast(DoubleType()))

training_2more_df.show(3)
validation_2more_df.show(3)
test_2more_df.show(3)

Training: 9416289, validation: 3140562, test: 3134102

+----------+----------+-----+
new_userId|new_songId|Plays|
+----------+----------+-----+
 2| 23720| 2|
 2| 130614| 3|
 2| 175180| 2|
+----------+----------+-----+
only showing top 3 rows

+----------+----------+-----+
new_userId|new_songId|Plays|
+----------+----------+-----+
 5| 3879| 2.0|
 5| 32786| 2.0|
 5| 151582| 2.0|
+----------+----------+-----+
only showing top 3 rows

+----------+----------+-----+
new_userId|new_songId|Plays|
+----------+----------+-----+
 4| 291935| 10.0|
 5| 88446| 8.0|
 5| 197575| 2.0|
+----------+----------+-----+
only showing top 3 rows

### Train Using Alternating Least Squares

In [59]:
# Let's initialize our ALS learner
als_2more = ALS()

# Now set the parameters for the method
als_2more.setMaxIter(2)\
   .setSeed(seed)\
   .setItemCol("new_songId")\
   .setRatingCol("Plays")\
   .setUserCol("new_userId")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="Plays", metricName="rmse")

tolerance = 0.03
ranks = [4, 8, 12, 16]
regParams = [0.1, 0.15, 0.2, 0.25]
errors = [[0]*len(ranks)]*len(regParams)
models = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1
i = 0
for regParam in regParams:
  j = 0
  for rank in ranks:
    # Set the rank here:
    als_2more.setParams(rank = rank, regParam = regParam)
    # Create the model with these parameters.
    model = als_2more.fit(training_2more_df)
    # Run the model to create a prediction. Predict against the validation_df.
    predict_df = model.transform(validation_2more_df)

    # Remove NaN values from prediction (due to SPARK-14489)
    predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
    predicted_plays_df = predicted_plays_df.withColumn("prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))
    # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
    error = reg_eval.evaluate(predicted_plays_df)
    errors[i][j] = error
    models[i][j] = model
    print ('For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error))
    if error < min_error:
      min_error = error
      best_params = [i,j]
    j += 1
  i += 1

als_2more.setRegParam(regParams[best_params[0]])
als_2more.setRank(ranks[best_params[1]])
print ('The best model was trained with regularization parameter %s' % regParams[best_params[0]])
print ('The best model was trained with rank %s' % ranks[best_params[1]])
my_model_2more = models[best_params[0]][best_params[1]]

For rank 4, regularization parameter 0.1 the RMSE is 13.184865519618292
For rank 8, regularization parameter 0.1 the RMSE is 12.73819243617186
For rank 12, regularization parameter 0.1 the RMSE is 12.016093328057908
For rank 16, regularization parameter 0.1 the RMSE is 11.748674123587799
For rank 4, regularization parameter 0.15 the RMSE is 11.944077870188465
For rank 8, regularization parameter 0.15 the RMSE is 11.384782117269651
For rank 12, regularization parameter 0.15 the RMSE is 10.79999919523712
For rank 16, regularization parameter 0.15 the RMSE is 10.621286376266848
For rank 4, regularization parameter 0.2 the RMSE is 11.248099852651647
For rank 8, regularization parameter 0.2 the RMSE is 10.705218383267567
For rank 12, regularization parameter 0.2 the RMSE is 10.268625324972696
For rank 16, regularization parameter 0.2 the RMSE is 10.147094632699774
For rank 4, regularization parameter 0.25 the RMSE is 10.832849575239294
For rank 8, regularization parameter 0.25 the RMSE is 10.336500847617668
For rank 12, regularization parameter 0.25 the RMSE is 10.006430097076828
For rank 16, regularization parameter 0.25 the RMSE is 9.922769320284214
The best model was trained with regularization parameter 0.25
The best model was trained with rank 16

In [60]:
# Let's initialize our ALS learner
als_2 = ALS()

# Now set the parameters for the method
als_2.setMaxIter(5)\
   .setSeed(seed)\
   .setItemCol("new_songId")\
   .setRatingCol("Plays")\
   .setUserCol("new_userId")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="Plays", metricName="rmse")

als_2.setParams(rank = 16, regParam = 0.25)
model_2 = als_2.fit(training_2more_df)
predict_2 = model_2.transform(validation_2more_df)
predict_2_test = model_2.transform(test_2more_df)

### Test the Model on the Test Dataset

In [62]:
predict2_df = model_2.transform(test_2more_df)

# Remove NaN values from prediction (due to SPARK-14489)
predicted2_test_df = predict2_df.filter(predict2_df.prediction != float('nan'))

# Round floats to whole numbers
predicted2_test_df = predicted2_test_df.withColumn("prediction", F.abs(F.round(predicted2_test_df["prediction"],0)))
# Run the previously created RMSE evaluator, reg_eval, on the predicted_test_df DataFrame
test2_RMSE = reg_eval.evaluate(predicted2_test_df)

print('The model had a RMSE on the test set of {0}'.format(test2_RMSE))

The model had a RMSE on the test set of 10.02409210286157

### Comparing the Model

We again compare to selecting the average number of plays from the training dataset

In [64]:
avg_plays2_df = training_2more_df.groupBy().avg('Plays').select(F.round('avg(Plays)'))

avg_plays2_df.show(3)
# Extract the average rating value. (This is row 0, column 0.)
training_avg_plays2 = avg_plays2_df.collect()[0][0]

print('The average number of plays in the dataset is {0}'.format(training_avg_plays2))

# Add a column with the average rating
test_for_avg2_df = test_2more_df.withColumn('prediction', F.lit(training_avg_plays2))

# Run the previously created RMSE evaluator, reg_eval, on the test_for_avg_df DataFrame
test_avg2_RMSE = reg_eval.evaluate(test_for_avg2_df)

print("The RMSE on the average set is {0}".format(test_avg2_RMSE))

+--------------------+
round(avg(Plays), 0)|
+--------------------+
 6.0|
+--------------------+

The average number of plays in the dataset is 6.0
The RMSE on the average set is 9.244973830567792

Though we see higher RMSEs than when we included songs with 1 play, this is to be expected since the model is likely to there is larger mean and larger variation.

We can also see that, compared to average number of plays in the dataset, excluding one play songs the model performs better.

### Prediction for a User

We use an example user to visually see the kind of songs the model predicts

In [67]:
listened_songs.select('artist_name', 'title').show()
unlistened_songs = raw_plays_df_with_int_ids.filter(~ raw_plays_df_with_int_ids['new_songId'].isin(listened_songs_list)) \
                                            .select('new_songId').withColumn('new_userId', F.lit(UserID)).distinct()

predicted_listens_2 = model_2.transform(unlistened_songs)
predicted_listens_2 = predicted_listens_2.filter(predicted_listens_2['prediction'] != float('nan'))

print('Predicted Songs:')
predicted_listens_2.join(raw_plays_df_with_int_ids, 'new_songId') \
                 .join(metadata_df, 'songId') \
                 .select('artist_name', 'title', 'prediction') \
                 .distinct() \
                 .orderBy('prediction', ascending = False) \
                 .show(10)

+-----------+--------------------+
artist_name| title|
+-----------+--------------------+
The Killers|A Dustland Fairytale|
+-----------+--------------------+

Predicted Songs:
+-------------------+--------------------+----------+
 artist_name| title|prediction|
+-------------------+--------------------+----------+
 Mary Black| Flesh And Blood| 40.482685|
 Mazz| Más Y Más| 33.126324|
 Back Door Slam|Heavy On My Mind ...| 30.418045|
 The Berzerker| Pure Hatred| 28.786285|
 Angela Dimitriou| Mi Mas Agapas| 26.135033|
My Chemical Romance|Interlude (Album ...| 26.135033|
 Planxty|Follow Me Up To C...| 26.038643|
 Olivia Ruiz| Enervé| 25.370657|
 Jeremy Spencer| Maria De Santiago| 24.715492|
 Prozak| Its Too Late Now| 24.189468|
+-------------------+--------------------+----------+
only showing top 10 rows

## Clustering -- K-Means Clustering for Song clusters -- Madhavi

In [69]:
# Step 1 join dataframes - append song hottness & artist hottness
df_for_clustering = raw_plays_df_with_int_ids.join(metadata_df.select('song_hotttnesss', 'artist_name', 'title','artist_familiarity', 'genre','artist_hotttness','songId'), 'songId')

#df_for_clustering = df_for_clustering.join
#df_for_clustering.na.fill(0).show(5)

In [70]:
from pyspark.sql import functions as F

df_temp = df_for_clustering.select(
  "artist_familiarity",
  F.col("artist_familiarity").cast("int").isNotNull().alias("Value")
)


df_temp.where(df_temp.Value == 'false').select("artist_familiarity").count()


Out[ 33 ]: 0

In [71]:
# Step2 - Assemble features
from pyspark.ml.feature import VectorAssembler

df = df_for_clustering
vecAssembler = VectorAssembler(inputCols=["Plays", "artist_hotttness"], outputCol="features")
new_df = vecAssembler.transform(df)
new_df.show(5)

+------------------+------------------+--------------------+-----+----------+----------+------------+----------------+-----------+-------------------+------------------+-----+------------------+--------------------+
 songId| artist_id| userId|Plays|new_userId|new_songId|new_artistId| song_hotttnesss|artist_name| title|artist_familiarity|genre| artist_hotttness| features|
+------------------+------------------+--------------------+-----+----------+----------+------------+----------------+-----------+-------------------+------------------+-----+------------------+--------------------+
SOCKUUJ12A6D4FA41C|ARXSABJ1187FB3C336|29ae33988946a423f...| 1| 786037| 9626| 301|0.69047353903576| Deftones|Lotion (LP Version)|0.8216973167394691| null|0.6017297557100237|[1.0,0.6017297557...|
SOCKUUJ12A6D4FA41C|ARXSABJ1187FB3C336|6866d8cf3f418f3dc...| 1| 783126| 9626| 301|0.69047353903576| Deftones|Lotion (LP Version)|0.8216973167394691| null|0.6017297557100237|[1.0,0.6017297557...|
SOCKUUJ12A6D4FA41C|ARXSABJ1187FB3C336|8ae7b3a604c68d688...| 25| 785407| 9626| 301|0.69047353903576| Deftones|Lotion (LP Version)|0.8216973167394691| null|0.6017297557100237|[25.0,0.601729755...|
SOCKUUJ12A6D4FA41C|ARXSABJ1187FB3C336|9b862d3237e93308b...| 1| 785441| 9626| 301|0.69047353903576| Deftones|Lotion (LP Version)|0.8216973167394691| null|0.6017297557100237|[1.0,0.6017297557...|
SOCKUUJ12A6D4FA41C|ARXSABJ1187FB3C336|b276add80e2341d59...| 1| 783535| 9626| 301|0.69047353903576| Deftones|Lotion (LP Version)|0.8216973167394691| null|0.6017297557100237|[1.0,0.6017297557...|
+------------------+------------------+--------------------+-----+----------+----------+------------+----------------+-----------+-------------------+------------------+-----+------------------+--------------------+
only showing top 5 rows

In [72]:
# Scale data
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')
scaler_model = scaler.fit(new_df)
scaled_data = scaler_model.transform(new_df)

In [73]:
scaled_data.show(5)

+------------------+------------------+--------------------+-----+----------+----------+------------+----------------+-----------+-------------------+------------------+-----+------------------+--------------------+--------------------+
 songId| artist_id| userId|Plays|new_userId|new_songId|new_artistId| song_hotttnesss|artist_name| title|artist_familiarity|genre| artist_hotttness| features| scaledFeatures|
+------------------+------------------+--------------------+-----+----------+----------+------------+----------------+-----------+-------------------+------------------+-----+------------------+--------------------+--------------------+
SOCKUUJ12A6D4FA41C|ARXSABJ1187FB3C336|29ae33988946a423f...| 1| 786037| 9626| 301|0.69047353903576| Deftones|Lotion (LP Version)|0.8216973167394691| null|0.6017297557100237|[1.0,0.6017297557...|[0.18332996073304...|
SOCKUUJ12A6D4FA41C|ARXSABJ1187FB3C336|6866d8cf3f418f3dc...| 1| 783126| 9626| 301|0.69047353903576| Deftones|Lotion (LP Version)|0.8216973167394691| null|0.6017297557100237|[1.0,0.6017297557...|[0.18332996073304...|
SOCKUUJ12A6D4FA41C|ARXSABJ1187FB3C336|8ae7b3a604c68d688...| 25| 785407| 9626| 301|0.69047353903576| Deftones|Lotion (LP Version)|0.8216973167394691| null|0.6017297557100237|[25.0,0.601729755...|[4.58324901832613...|
SOCKUUJ12A6D4FA41C|ARXSABJ1187FB3C336|9b862d3237e93308b...| 1| 785441| 9626| 301|0.69047353903576| Deftones|Lotion (LP Version)|0.8216973167394691| null|0.6017297557100237|[1.0,0.6017297557...|[0.18332996073304...|
SOCKUUJ12A6D4FA41C|ARXSABJ1187FB3C336|b276add80e2341d59...| 1| 783535| 9626| 301|0.69047353903576| Deftones|Lotion (LP Version)|0.8216973167394691| null|0.6017297557100237|[1.0,0.6017297557...|[0.18332996073304...|
+------------------+------------------+--------------------+-----+----------+----------+------------+----------------+-----------+-------------------+------------------+-----+------------------+--------------------+--------------------+
only showing top 5 rows

In [74]:
# Step 2 - fit  KMeans model
from pyspark.ml.clustering import KMeans

In [75]:
k_means_10 = KMeans(featuresCol='scaledFeatures', k=10)
k_means_15 = KMeans(featuresCol='scaledFeatures', k=15)
model_k10 = k_means_10.fit(scaled_data)
model_k15 = k_means_15.fit(scaled_data)

In [76]:
model_k15_data = model_k15.transform(scaled_data)
#model_k15_data.groupBy('prediction').count().show()


In [77]:
UserID = 177833
listened_songs_cl = model_k15_data.filter(model_k15_data.new_userId == UserID) \
                                          .select('new_songId', 'artist_name', 'title','Plays','Prediction')  
                                          


listened_songs_cl.show()


+----------+---------------+--------------------+-----+----------+
new_songId| artist_name| title|Plays|Prediction|
+----------+---------------+--------------------+-----+----------+
 107445| Less Than Jake| Automatic| 2| 0|
 171660|Andres Calamaro|Un vestido y un a...| 1| 0|
 195260| 2 Minutos| 2 Minutos| 1| 1|
 253574| Bob Marley| Lively Up Yourself| 2| 0|
 278808| Fito Paez| Tercer Mundo| 1| 10|
 315736| Goldfinger| I'm Down| 2| 0|
 328694| Radiohead|Everything In Its...| 1| 11|
 336833| Goldfinger|Ocean Size (Album...| 2| 0|
 336949| 2 Minutos| Rompan Todo| 2| 1|
 351986| Radiohead| 15 Step| 1| 11|
+----------+---------------+--------------------+-----+----------+

In [78]:
UserID = 3543
listened_songs_cl = model_k15_data.filter(model_k15_data.new_userId == UserID) \
                                          .select('new_songId', 'artist_name', 'title','Plays','Prediction')  
                                          


listened_songs_cl.show()

+----------+-----------+-----+-----+----------+
new_songId|artist_name|title|Plays|Prediction|
+----------+-----------+-----+-----+----------+
+----------+-----------+-----+-----+----------+

In [79]:
# Find other songs in the same cluster
other_songs_cl0 = model_k15_data.filter(model_k15_data.prediction == 0) \
                                          .select('new_songId', 'artist_name', 'title','Plays','Prediction')  
                                          

  # Find other songs in the same cluster
other_songs_cl11 = model_k15_data.filter(model_k15_data.prediction == 11) \
                                          .select('new_songId', 'artist_name', 'title','Plays','Prediction')  

In [80]:
# Other songs in the same cluster
other_songs_cl0.toPandas().drop_duplicates().sort_values('Plays', ascending=False ).head(5)


,new_songId,artist_name,title,Plays,Prediction
135356,305745,The Crystal Method,Keep Hope Alive,3,0
108615,257480,The Cardigans,Iron Man,3,0
45924,163994,The Presets,Are You The One?,3,0
163481,356041,The Misfits,Halloween II,3,0
146655,319127,Phil Collins,Two Hearts,3,0


In [81]:
other_songs_cl11.toPandas().drop_duplicates().sort_values('Plays', ascending=False ).head(5)

,new_songId,artist_name,title,Plays,Prediction
50349,184410,Kelly Clarkson,Cry,5,11
97579,290467,Madonna,Papa Don't Preach,5,11
62673,253709,The White Stripes,Im Slowly Turning Into You,5,11
111546,332564,Jack Johnson,Monsoon,5,11
47464,171211,Radiohead,Fake Plastic Trees (Acoustic Version),5,11


In [82]:
other_songs_cl9 = model_k15_data.filter(model_k15_data.prediction == 9) \
                                          .select('new_songId', 'artist_name', 'title','Plays','Prediction') 
other_songs_cl9.toPandas().drop_duplicates().sort_values('Plays', ascending=False ).head(5)

,new_songId,artist_name,title,Plays,Prediction
94957,300455,Ziggy Marley And The Melody Makers,Namibia,3,9
85706,257635,Plies,I Am The Club (Explicit Album Version),3,9
90517,50061,Lupe Fiasco feat. GemStones,Go Baby (feat GemStones) (Explicit Album Version),3,9
89323,92454,Lupe Fiasco feat. Matthew Santos,Fighters (feat. Matthew Santos) (Amended Album...,3,9
89305,300018,Lupe Fiasco feat. Matthew Santos,Fighters (feat. Matthew Santos) (Amended Album...,3,9


In [83]:
other_songs_cl1 = model_k15_data.filter(model_k15_data.prediction == 1) \
                                          .select('new_songId', 'artist_name', 'title','Plays','Prediction') 
other_songs_cl1.toPandas().drop_duplicates().sort_values('Plays', ascending=False ).head(5)

,new_songId,artist_name,title,Plays,Prediction
9414,88189,Usher Featuring The Nu Beginning,Only Human,6,1
9119,151644,Usher Featuring The Nu Beginning,Only Human,6,1
17515,51993,Usher,Hey Daddy (Daddy's Home),6,1
17402,23145,Usher,Hey Daddy (Daddy's Home),6,1
60803,340735,Usher,I'll Show You Love,6,1


## Clustering -- K-Means Clustering Elbow Plot -- Annie

In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets.samples_generator import make_blobs
from pyspark import SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext

### Optimize choice of k

One disadvantage of KMeans compared to more advanced clustering algorithms is that the algorithm must be told how many clusters, k, it should try to find. To optimize k we cluster a fraction of the data for different choices of k and look for an "elbow" in the cost function.

In [87]:
cost = np.zeros(20)
for k in range(2,20):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(new_df.sample(False,0.1, seed=42))
    cost[k] = model.computeCost(new_df)

In [88]:
fig, ax = plt.subplots(1,1, figsize =(10,8))
ax.plot(range(2,20),cost[2:20])
ax.set_xlabel('k')
ax.set_ylabel('cost')
display(fig)

Look like there is very little gain after k=15, so we stick to that choice when processing the full data set.